In [152]:
from power_outage_data import load_yearly_data, load_fips_shapes, lat_lon_to_fips
from storm_data import (
    load_clusters,
    load_tracks,
    date_str_to_storm_time,
    storm_time_to_datetime,
    get_intensity
)

yearly_power_data = load_yearly_data()
fips_shapes = load_fips_shapes()
clusters = load_clusters()
# Load tracks and prefilter for year.
tks = load_tracks()
tks = tks.where(tks.season>=2014, drop=True)

In [153]:
# iterate over each cluster

def lookup_storms_in_cluster(cluster):
    storm_dicts = []
    for _, row in cluster.iterrows():
        d = row.to_dict()
        start = date_str_to_storm_time(d['first_landfall_time']) - 0.01
        end = date_str_to_storm_time(d['final_landfall_time']) + 0.01
        sid = bytes(d['sid'], 'utf-8')

        storms = tks.where(
            (tks['time'] >= start) & (tks['time'] <= end) & (tks['sid'] == sid),
            drop=True
        )
        storm = storms.sel(storm=0)
        lon = storm.lon.values
        lat = storm.lat.values
        fips_codes = []
        county = []
        state = []
        for lat, lon in zip(lat, lon):
            fips = lat_lon_to_fips(lat, lon, fips_shapes)
            if fips:
                fips_codes.append(fips['id'])
                county.append(fips['properties']['NAME'])
                state.append(fips['properties']['STATE'])
            else:
                fips_codes.append(None)
                county.append(None)
                state.append(None)

        storm_dict = {
            **d,
            'year': int(storm.season.values[0]),
            'intensity': get_intensity(storm),
            'times': [storm_time_to_datetime(time) for time in storm.time.values],
            'lon': storm.lon.values,
            'lat': storm.lat.values,
            'fips_code': fips_codes,
            'county': county
        }
        storm_dicts.append(storm_dict)

    return storm_dicts


cluster_0 = list(clusters)[0][1]

cluster_0_storms = lookup_storms_in_cluster(cluster_0)
cluster_0_storms[1]


{'storm': 21,
 'sid': '2019299N25265',
 'name': 'OLGA',
 'fullmoment_label': 0,
 'spatmoment_label': 0,
 'first_landfall_time': '2019-10-26 06:00:00',
 'final_landfall_time': '2019-10-27 15:00:00',
 'year': 2019,
 'intensity': np.float32(45.0),
 'times': [datetime.datetime(2019, 10, 26, 6, 0),
  datetime.datetime(2019, 10, 26, 9, 0),
  datetime.datetime(2019, 10, 26, 12, 0),
  datetime.datetime(2019, 10, 26, 15, 0),
  datetime.datetime(2019, 10, 26, 18, 0),
  datetime.datetime(2019, 10, 26, 21, 0),
  datetime.datetime(2019, 10, 27, 0, 0),
  datetime.datetime(2019, 10, 27, 3, 0),
  datetime.datetime(2019, 10, 27, 6, 0),
  datetime.datetime(2019, 10, 27, 9, 0),
  datetime.datetime(2019, 10, 27, 12, 0),
  datetime.datetime(2019, 10, 27, 15, 0)],
 'lon': array([-91.200005, -90.554405, -90.      , -89.65197 , -89.3     ,
        -88.67649 , -88.      , -87.565056, -87.      , -86.10014 ,
        -84.7     , -82.69062 ], dtype=float32),
 'lat': array([28.8     , 30.048195, 31.7     , 33.8744

# Visualize a cluster

In [ ]:
import pandas as pd

storm = cluster_0_storms[0]
power_outage_data = yearly_power_data[storm['year']]

storm_df = pd.DataFrame({
    'fips_code': storm['fips_code'],
    'date': [time.date() for time in storm['times']],
    'hour': [time.hour for time in storm['times']],
    'lat': storm['lat'],
    'lon': storm['lon'],
})
storm_df
merged  = storm_df.merge(power_outage_data.drop(columns='hour'), on=['fips_code', 'date'], how='left')
merged



,fips_code,day,hour,lat,lon,county,state,customers_out
0,None,2018-05-28,18,29.799997,-85.900002,NaN,NaN,NaN
1,12131,2018-05-28,21,30.299997,-86.000000,Walton,Florida,563.0
2,12131,2018-05-29,0,30.900000,-86.099998,Walton,Florida,535.0
3,01039,2018-05-29,3,31.413733,-86.326035,Covington,Alabama,235.0
4,01013,2018-05-29,6,31.900000,-86.599998,Butler,Alabama,1310.0
5,01001,2018-05-29,9,32.434963,-86.815224,Autauga,Alabama,2556.0
6,01007,2018-05-29,12,33.000000,-87.000000,Bibb,Alabama,493.0
7,01073,2018-05-29,15,33.592453,-87.157814,Jefferson,Alabama,11355.0
8,01133,2018-05-29,18,34.200001,-87.300003,Winston,Alabama,581.0
9,01033,2018-05-29,21,34.792480,-87.450264,NaN,NaN,NaN
